##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TFX 및 Vertex 파이프라인을 사용하여 BigQuery에서 데이터 읽기


<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td><a target="_blank" href="https://www.tensorflow.org/tfx/tutorials/tfx/gcp/vertex_pipelines_bq"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서 보기</a></td>
<td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ko/tfx/tutorials/tfx/gcp/vertex_pipelines_bq.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행하기</a> </td>
<td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ko/tfx/tutorials/tfx/gcp/vertex_pipelines_bq.ipynb"><img width="32px" src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub에서 소스 보기</a></td>
<td>     <a href="https://storage.googleapis.com/tensorflow_docs/tfx/docs/tutorials/tfx/gcp/vertex_pipelines_bq.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드하기</a> </td>
<td><a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?q=download_url%3Dhttps://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ko/tfx/tutorials/tfx/gcp/vertex_pipelines_bq.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Google Cloud Vertex AI Workbench에서 실행하기</a></td>
</table></div>


이 노트북 기반 튜토리얼에서는 [Google Cloud BigQuery](https://cloud.google.com/bigquery)를 ML 모델을 학습시키기 위한 데이터 소스로 사용합니다. ML 파이프라인은 TFX를 사용하여 구성되고 Google Cloud Vertex Pipelines에서 실행됩니다.

이 노트북은 [Vertex 파이프라인용 단순 TFX 파이프라인 튜토리얼](https://www.tensorflow.org/tfx/tutorials/tfx/gcp/vertex_pipelines_simple)에서 구축한 TFX 파이프라인을 기반으로 합니다. 해당 튜토리얼을 아직 읽지 않았다면 이 노트북을 계속 진행하기 전에 꼭 읽기 바랍니다.

[BigQuery](https://cloud.google.com/bigquery)는 비즈니스 민첩성을 위해 설계된 확장성이 뛰어나고 비용 효율적인 서버리스 멀티 클라우드 데이터 웨어하우스입니다. TFX를 사용하여 BigQuery에서 학습 데이터를 읽고 [학습된 모델을 BigQuery에 게시](https://www.tensorflow.org/tfx/api_docs/python/tfx/v1/extensions/google_cloud_big_query/Pusher)할 수 있습니다.

이 튜토리얼에서는 BigQuery에서 TFX 파이프라인으로 데이터를 읽는 `BigQueryExampleGen` 구성 요소를 사용합니다.


이 노트북은 [Google Colab](https://colab.research.google.com/notebooks/intro.ipynb) 또는 [AI Platform Notebooks](https://cloud.google.com/ai-platform-notebooks)에서 실행하기 위한 것입니다. 이 중 하나도 사용하지 않는 경우, 위의 "Google Colab에서 실행하기" 버튼을 클릭하면 됩니다.

## 설정

[Vertex 파이프라인용 단순 TFX 파이프라인 튜토리얼](https://www.tensorflow.org/tfx/tutorials/tfx/gcp/vertex_pipelines_simple)을 마쳤다면 실제 작동하는 GCP 프로젝트와 GCS 버킷이 있을 것이며 이것이 이 튜토리얼에 필요한 전부입니다. 예비 튜토리얼을 아직 보지 않았다면 먼저 읽어보세요.

### Python 패키지 설치하기

TFX 및 KFP를 포함한 필수 Python 패키지를 설치하여 ML 파이프라인을 작성하고 Vertex 파이프라인에 작업을 제출합니다.

In [ ]:
# Use the latest version of pip.
!pip install --upgrade pip
!pip install --upgrade "tfx[kfp]<2"

### shapely 설치 제거하기

TODO(b/263441833) ImportError를 피하는 임시 솔루션입니다. 다른 추가 종속성을 제거하는 대신 최신 버전의 Bigquery를 지원하여 처리하는 것이 이상적입니다.

In [ ]:
!pip uninstall shapely -y

#### 런타임을 다시 시작했습니까?

Google Colab을 사용하는 경우, "런타임 다시 시작" 버튼을 클릭하거나 "런타임 &gt; 렁타임 다시 시작..." 메뉴를 사용하여 런타임을 다시 시작해야 합니다. 이는 Colab이 패키지를 로드하는 방식 때문입니다.

Colab에 있지 않다면 다음 셀을 사용하여 런타임을 다시 시작할 수 있습니다.

In [ ]:
# docs_infra: no_execute
import sys
if not 'google.colab' in sys.modules:
  # Automatically restart kernel after installs
  import IPython
  app = IPython.Application.instance()
  app.kernel.do_shutdown(True)

### 이 노트북을 사용하려면 Google에 로그인하세요

Colab에서 이 노트북을 실행하는 경우 사용자 계정으로 인증하세요.

In [ ]:
import sys
if 'google.colab' in sys.modules:
  from google.colab import auth
  auth.authenticate_user()

**AI Platform Notebooks를 사용 중인 경우** 이어지는 섹션을 실행하기 전에 다음을 실행하여 Google Cloud에 인증합니다.

```sh
gcloud auth login
```

**터미널 창**(메뉴에서 **파일** &gt; **새로 만들기**를 통해 열 수 있음)에서 실행하면 됩니다. 이 작업은 노트북 인스턴스당 한 번만 수행하면 됩니다.

패키지 버전을 확인합니다.

In [ ]:
import tensorflow as tf
print('TensorFlow version: {}'.format(tf.__version__))
from tfx import v1 as tfx
print('TFX version: {}'.format(tfx.__version__))
import kfp
print('KFP version: {}'.format(kfp.__version__))

### 변수 설정하기

아래에서 파이프라인을 사용자 지정하는 데 사용되는 몇 가지 변수를 설정합니다. 다음 정보가 필요합니다.

- GCP 프로젝트 ID 및 번호. [프로젝트 ID 및 번호 확인](https://cloud.google.com/resource-manager/docs/creating-managing-projects#identifying_projects)을 참조하세요.
- 파이프라인을 실행할 GCP 영역. Vertex 파이프라인을 사용할 수 있는 영역에 대한 자세한 내용은 [Vertex AI 위치 가이드](https://cloud.google.com/vertex-ai/docs/general/locations#feature-availability)를 참조하세요.
- 파이프라인 출력을 저장하기 위한 Google Cloud Storage 버킷

**실행하기 전에 아래 셀에 필요한 값을 입력하세요**.


In [ ]:
GOOGLE_CLOUD_PROJECT = ''         # <--- ENTER THIS
GOOGLE_CLOUD_PROJECT_NUMBER = ''  # <--- ENTER THIS
GOOGLE_CLOUD_REGION = ''          # <--- ENTER THIS
GCS_BUCKET_NAME = ''              # <--- ENTER THIS

if not (GOOGLE_CLOUD_PROJECT and  GOOGLE_CLOUD_PROJECT_NUMBER and GOOGLE_CLOUD_REGION and GCS_BUCKET_NAME):
    from absl import logging
    logging.error('Please set all required parameters.')

프로젝트를 사용하도록 `gcloud`를 설정합니다.

In [ ]:
!gcloud config set project {GOOGLE_CLOUD_PROJECT}

In [ ]:
PIPELINE_NAME = 'penguin-bigquery'

# Path to various pipeline artifact.
PIPELINE_ROOT = 'gs://{}/pipeline_root/{}'.format(
    GCS_BUCKET_NAME, PIPELINE_NAME)

# Paths for users' Python module.
MODULE_ROOT = 'gs://{}/pipeline_module/{}'.format(
    GCS_BUCKET_NAME, PIPELINE_NAME)

# Paths for users' data.
DATA_ROOT = 'gs://{}/data/{}'.format(GCS_BUCKET_NAME, PIPELINE_NAME)

# This is the path where your model will be pushed for serving.
SERVING_MODEL_DIR = 'gs://{}/serving_model/{}'.format(
    GCS_BUCKET_NAME, PIPELINE_NAME)

print('PIPELINE_ROOT: {}'.format(PIPELINE_ROOT))

기본적으로 Vertex 파이프라인은 `[project-number]-compute@developer.gserviceaccount.com` 형식의 기본 GCE VM 서비스 계정을 사용합니다. 파이프라인에서 BigQuery에 액세스하려면 이 계정에 BigQuery를 사용할 수 있는 권한을 부여해야 합니다. 계정에 'BigQuery 사용자' 역할을 추가합니다.

In [ ]:
!gcloud projects add-iam-policy-binding {GOOGLE_CLOUD_PROJECT} \
  --member=serviceAccount:{GOOGLE_CLOUD_PROJECT_NUMBER}-compute@developer.gserviceaccount.com \
  --role=roles/bigquery.user

서비스 계정 및 IAM 구성에 대한 자세한 내용은 [Vertex 설명서](https://cloud.google.com/vertex-ai/docs/pipelines/configure-project)를 참조하세요.

## 파이프라인 생성하기

TFX 파이프라인은 [Vertex 파이프라인용 단순 TFX 파이프라인 튜토리얼](https://www.tensorflow.org/tfx/tutorials/tfx/gcp/vertex_pipelines_simple)에서 했던 것처럼 Python API를 사용하여 정의됩니다. 이전에는 CSV 파일에서 데이터를 읽는 `CsvExampleGen`을 사용했습니다. 이 튜토리얼에서는 BigQuery에서 데이터를 읽는 [`BigQueryExampleGen`](https://www.tensorflow.org/tfx/api_docs/python/tfx/v1/extensions/google_cloud_big_query/BigQueryExampleGen) 구성 요소를 사용합니다.


### BigQuery 쿼리 준비하기

동일한 [Palmer Penguins 데이터세트](https://allisonhorst.github.io/palmerpenguins/articles/intro.html)를 사용할 것입니다. 그러나 동일한 CSV 파일을 사용하여 채워진 BigQuery 테이블 `tfx-oss-public.palmer_penguins.palmer_penguins`에서 이를 읽습니다.

Google Colab을 사용하는 경우 BigQuery 테이블의 내용을 직접 검사할 수 있습니다.

In [ ]:
# docs_infra: no_execute
%%bigquery --project {GOOGLE_CLOUD_PROJECT}
SELECT *
FROM `tfx-oss-public.palmer_penguins.palmer_penguins`
LIMIT 5

레이블인 `species`을 제외한 모든 특성은 이미 0~1로 정규화되었습니다. 펭귄의 `species`을 예측하는 분류 모델을 구축할 것입니다.

`BigQueryExampleGen`에는 가져올 데이터를 지정하는 쿼리가 필요합니다. 테이블에 있는 모든 행의 모든 필드를 사용하기 때문에 쿼리는 매우 간단합니다. <a>BigQuery 표준 SQL 구문</a>에 따라 필요 시 필드 이름을 지정하고 <code>WHERE</code> 조건을 추가할 수도 있습니다.

In [ ]:
QUERY = "SELECT * FROM `tfx-oss-public.palmer_penguins.palmer_penguins`"

### 모델 코드를 작성합니다.

[단순 TFX 파이프라인 튜토리얼](https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple)에서와 동일한 모델 코드를 사용합니다.

In [ ]:
_trainer_module_file = 'penguin_trainer.py'

In [ ]:
%%writefile {_trainer_module_file}

# Copied from https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple

from typing import List
from absl import logging
import tensorflow as tf
from tensorflow import keras
from tensorflow_transform.tf_metadata import schema_utils

from tfx import v1 as tfx
from tfx_bsl.public import tfxio

from tensorflow_metadata.proto.v0 import schema_pb2

_FEATURE_KEYS = [
    'culmen_length_mm', 'culmen_depth_mm', 'flipper_length_mm', 'body_mass_g'
]
_LABEL_KEY = 'species'

_TRAIN_BATCH_SIZE = 20
_EVAL_BATCH_SIZE = 10

# Since we're not generating or creating a schema, we will instead create
# a feature spec.  Since there are a fairly small number of features this is
# manageable for this dataset.
_FEATURE_SPEC = {
    **{
        feature: tf.io.FixedLenFeature(shape=[1], dtype=tf.float32)
           for feature in _FEATURE_KEYS
       },
    _LABEL_KEY: tf.io.FixedLenFeature(shape=[1], dtype=tf.int64)
}


def _input_fn(file_pattern: List[str],
              data_accessor: tfx.components.DataAccessor,
              schema: schema_pb2.Schema,
              batch_size: int) -> tf.data.Dataset:
  """Generates features and label for training.

  Args:
    file_pattern: List of paths or patterns of input tfrecord files.
    data_accessor: DataAccessor for converting input to RecordBatch.
    schema: schema of the input data.
    batch_size: representing the number of consecutive elements of returned
      dataset to combine in a single batch

  Returns:
    A dataset that contains (features, indices) tuple where features is a
      dictionary of Tensors, and indices is a single Tensor of label indices.
  """
  return data_accessor.tf_dataset_factory(
      file_pattern,
      tfxio.TensorFlowDatasetOptions(
          batch_size=batch_size, label_key=_LABEL_KEY),
      schema=schema).repeat()


def _make_keras_model() -> tf.keras.Model:
  """Creates a DNN Keras model for classifying penguin data.

  Returns:
    A Keras Model.
  """
  # The model below is built with Functional API, please refer to
  # https://www.tensorflow.org/guide/keras/overview for all API options.
  inputs = [keras.layers.Input(shape=(1,), name=f) for f in _FEATURE_KEYS]
  d = keras.layers.concatenate(inputs)
  for _ in range(2):
    d = keras.layers.Dense(8, activation='relu')(d)
  outputs = keras.layers.Dense(3)(d)

  model = keras.Model(inputs=inputs, outputs=outputs)
  model.compile(
      optimizer=keras.optimizers.Adam(1e-2),
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=[keras.metrics.SparseCategoricalAccuracy()])

  model.summary(print_fn=logging.info)
  return model


# TFX Trainer will call this function.
def run_fn(fn_args: tfx.components.FnArgs):
  """Train the model based on given args.

  Args:
    fn_args: Holds args used to train the model as name/value pairs.
  """

  # This schema is usually either an output of SchemaGen or a manually-curated
  # version provided by pipeline author. A schema can also derived from TFT
  # graph if a Transform component is used. In the case when either is missing,
  # `schema_from_feature_spec` could be used to generate schema from very simple
  # feature_spec, but the schema returned would be very primitive.
  schema = schema_utils.schema_from_feature_spec(_FEATURE_SPEC)

  train_dataset = _input_fn(
      fn_args.train_files,
      fn_args.data_accessor,
      schema,
      batch_size=_TRAIN_BATCH_SIZE)
  eval_dataset = _input_fn(
      fn_args.eval_files,
      fn_args.data_accessor,
      schema,
      batch_size=_EVAL_BATCH_SIZE)

  model = _make_keras_model()
  model.fit(
      train_dataset,
      steps_per_epoch=fn_args.train_steps,
      validation_data=eval_dataset,
      validation_steps=fn_args.eval_steps)

  # The result of the training should be saved in `fn_args.serving_model_dir`
  # directory.
  model.save(fn_args.serving_model_dir, save_format='tf')

파이프라인 구성 요소에서 액세스할 수 있는 GCS에 모듈 파일을 복사합니다. 모델 학습은 GCP에서 이루어지므로 이 모델 정의를 업로드해야 합니다.

그렇지 않으면 모듈 파일을 포함하는 컨테이너 이미지를 빌드하고 이미지를 사용하여 파이프라인을 실행할 수 있습니다.

In [ ]:
!gsutil cp {_trainer_module_file} {MODULE_ROOT}/

### 파이프라인 정의 작성하기

TFX 파이프라인을 생성하는 함수를 정의합니다. `query`를 인수로 사용하는 `BigQueryExampleGen`을 사용해야 합니다. 이전 튜토리얼에서 한 가지 더 변경된 점은 실행 시 구성 요소에 전달되는 `beam_pipeline_args`를 전달해야 한다는 것입니다. `beam_pipeline_args`를 사용하여 BigQuery에 추가 매개변수를 전달합니다.


In [ ]:
from typing import List, Optional

def _create_pipeline(pipeline_name: str, pipeline_root: str, query: str,
                     module_file: str, serving_model_dir: str,
                     beam_pipeline_args: Optional[List[str]],
                     ) -> tfx.dsl.Pipeline:
  """Creates a TFX pipeline using BigQuery."""

  # NEW: Query data in BigQuery as a data source.
  example_gen = tfx.extensions.google_cloud_big_query.BigQueryExampleGen(
      query=query)

  # Uses user-provided Python function that trains a model.
  trainer = tfx.components.Trainer(
      module_file=module_file,
      examples=example_gen.outputs['examples'],
      train_args=tfx.proto.TrainArgs(num_steps=100),
      eval_args=tfx.proto.EvalArgs(num_steps=5))

  # Pushes the model to a file destination.
  pusher = tfx.components.Pusher(
      model=trainer.outputs['model'],
      push_destination=tfx.proto.PushDestination(
          filesystem=tfx.proto.PushDestination.Filesystem(
              base_directory=serving_model_dir)))

  components = [
      example_gen,
      trainer,
      pusher,
  ]

  return tfx.dsl.Pipeline(
      pipeline_name=pipeline_name,
      pipeline_root=pipeline_root,
      components=components,
      # NEW: `beam_pipeline_args` is required to use BigQueryExampleGen.
      beam_pipeline_args=beam_pipeline_args)

## Vertex 파이프라인에서 파이프라인을 실행합니다

[Vertex 파이프라인용 단순 TFX 파이프라인 튜토리얼](https://www.tensorflow.org/tfx/tutorials/tfx/gcp/vertex_pipelines_simple)에서 했던 것처럼 Vertex 파이프라인을 사용하여 파이프라인을 실행합니다.


또한 BigQueryExampleGen에 대해 `beam_pipeline_args`를 전달해야 합니다. 여기에는 GCP 프로젝트 이름 및 BigQuery 실행을 위한 임시 저장소와 같은 구성이 포함됩니다.

In [ ]:
# docs_infra: no_execute
import os

# We need to pass some GCP related configs to BigQuery. This is currently done
# using `beam_pipeline_args` parameter.
BIG_QUERY_WITH_DIRECT_RUNNER_BEAM_PIPELINE_ARGS = [
   '--project=' + GOOGLE_CLOUD_PROJECT,
   '--temp_location=' + os.path.join('gs://', GCS_BUCKET_NAME, 'tmp'),
   ]

PIPELINE_DEFINITION_FILE = PIPELINE_NAME + '_pipeline.json'

runner = tfx.orchestration.experimental.KubeflowV2DagRunner(
    config=tfx.orchestration.experimental.KubeflowV2DagRunnerConfig(),
    output_filename=PIPELINE_DEFINITION_FILE)
_ = runner.run(
    _create_pipeline(
        pipeline_name=PIPELINE_NAME,
        pipeline_root=PIPELINE_ROOT,
        query=QUERY,
        module_file=os.path.join(MODULE_ROOT, _trainer_module_file),
        serving_model_dir=SERVING_MODEL_DIR,
        beam_pipeline_args=BIG_QUERY_WITH_DIRECT_RUNNER_BEAM_PIPELINE_ARGS))

생성된 정의 파일은 kfp 클라이언트를 사용하여 제출할 수 있습니다.

In [ ]:
# docs_infra: no_execute
from google.cloud import aiplatform
from google.cloud.aiplatform import pipeline_jobs
import logging
logging.getLogger().setLevel(logging.INFO)

aiplatform.init(project=GOOGLE_CLOUD_PROJECT, location=GOOGLE_CLOUD_REGION)

job = pipeline_jobs.PipelineJob(template_path=PIPELINE_DEFINITION_FILE,
                                display_name=PIPELINE_NAME)
job.submit()

이제 위의 출력에 있는 링크를 방문하거나 [Google Cloud Console](https://console.cloud.google.com/)에서 'Vertex AI &gt; 파이프라인'을 방문하여 진행 상황을 확인할 수 있습니다.